In [214]:
# Include necessary imports
import os
import torch 
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from torch.nn.utils.rnn import pad_sequence
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random

In [227]:
# Preprocess the data

# Transpose all valies to the same key
def key_transposition(df):
    transpose_val = 48 - df.iloc[3][0]
    df += transpose_val
    return df

# Min-Max normalization technique
def normalize_df(df):
    X_std = df / 127
    return X_std

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                transposed_df = key_transposition(df.transpose())
                normalized_df = normalize_df(transposed_df)
                if dirname == 'test':
                    test.append(normalized_df)
                if dirname == 'train':
                    train.append(normalized_df)
                if dirname == 'valid':
                    validation.append(normalized_df)

# Model

In [403]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=40, n_layers=2, dropout_rate=.3):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout_rate)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        self.dropout = torch.nn.Dropout(dropout_rate)
#         self.sigmoid = torch.nn.Sigmoid()
#         self.soft_max = torch.nn.Softmax(dim=0)
#         self.soft_max = torch.nn.Softmax(dim=-1)

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device),
            torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device))
        
    def forward(self, x, hidden=None, target=None, teacher_forcing=.7):
        batch_size, seq_len, _ = x.size()
#         if hidden is None:
        hidden = self.init_hidden(batch_size)
        lstm_input = x[:, 0:1, :]

        outputs = []
        for t in range(seq_len):
            lstm_output, hidden = self.lstm(lstm_input, hidden)
            lstm_output_step = lstm_output[:, -1, :] # isolate the step
            model_output_step = self.fc(lstm_output_step)
            outputs.append(model_output_step.unsqueeze(1))
            
            if target is not None and random.random() < teacher_forcing:
                next_harmony = target[:, t:t+1, :]
            else:
                next_harmony = model_output_step.unsqueeze(1)
            
            if t + 1 < seq_len:
                melody_at_next_step = x[:, t+1:t+2, :1]  # Extract the melody for the next step
                lstm_input = torch.cat((melody_at_next_step, next_harmony), dim=2)
        return torch.cat(outputs, dim=1) 

# Train

In [406]:
def train_model(model, optimizer, criterion, num_epochs):
    model.train()
    for song_index, song in enumerate(train[:15]):
        print(f"Training on song {song_index + 1}")
        
        melody = torch.tensor(song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, song.shape[1], 1)
        harmonies = torch.tensor(song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
#         input_song = torch.tensor(song.values.T, dtype=torch.float32).unsqueeze(0)
        harmonies_with_zero = torch.zeros(1, song.shape[1], 3)
        melody_with_empty_harmonies = torch.cat((melody, harmonies_with_zero), dim = -1)
       
        
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            output = model(melody_with_empty_harmonies, harmonies)
            output = output.reshape(-1, 3)

#             print("AFTER: ", inverse_df(output))
            harmonies = harmonies.reshape(-1, 3)
            loss = criterion(output, harmonies)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f"Song {song_index + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
                if torch.isnan(loss):
                    print("harmonies: ", harmonies)
                    print("melody:" , melody)
                    print("output: ", output)
                    

        model.eval()
        total_val_loss = 0
        
        # validation songs
        with torch.no_grad():
            for val_song in validation:
                val_melody = torch.tensor(val_song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, val_song.shape[1], 1)
                val_harmonies = torch.tensor(val_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
                val_harmonies_with_zero = torch.zeros(1, val_song.shape[1], 3)
                val_melody_with_empty_harmonies = torch.cat((val_melody, val_harmonies_with_zero), dim = -1)
                
                val_output = model(val_melody_with_empty_harmonies)
                val_output = val_output.reshape(-1, 3)
                val_harmonies = val_harmonies.reshape(-1, 3)
                
                val_loss = criterion(val_output, val_harmonies)
                total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(validation)
        print(f"Validation Loss after song {song_index + 1}: {average_val_loss}")

In [407]:
criterion = torch.nn.CrossEntropyLoss()
num_epochs = 100

model = Model(4, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train_model(model, optimizer, criterion, num_epochs)

Training on song 1
Song 1, Epoch 10/100, Loss: 1.3789119720458984
Song 1, Epoch 20/100, Loss: 1.377606987953186
Song 1, Epoch 30/100, Loss: 1.376392126083374
Song 1, Epoch 40/100, Loss: 1.375434398651123
Song 1, Epoch 50/100, Loss: 1.3746064901351929
Song 1, Epoch 60/100, Loss: 1.3738569021224976
Song 1, Epoch 70/100, Loss: 1.373364806175232
Song 1, Epoch 80/100, Loss: 1.3729602098464966
Song 1, Epoch 90/100, Loss: 1.3727221488952637
Song 1, Epoch 100/100, Loss: 1.3725099563598633
Validation Loss after song 1: 1.41918949286143
Training on song 2
Song 2, Epoch 10/100, Loss: 1.3615692853927612
Song 2, Epoch 20/100, Loss: 1.361404299736023
Song 2, Epoch 30/100, Loss: 1.3613603115081787
Song 2, Epoch 40/100, Loss: 1.3613561391830444
Song 2, Epoch 50/100, Loss: 1.3613553047180176
Song 2, Epoch 60/100, Loss: 1.3613542318344116
Song 2, Epoch 70/100, Loss: 1.3613531589508057
Song 2, Epoch 80/100, Loss: 1.3613526821136475
Song 2, Epoch 90/100, Loss: 1.3613522052764893
Song 2, Epoch 100/100, Los

# Hyperparameter Tuning

In [ ]:
learning = [0.01, .001]
n_layers= [1,2,3]
hidden_dim = [20, 40, 50]
epochs= [5000, 10000]
best_loss = float('inf')
best_params = {}

for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            for dims in hidden_dim:
                print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch} and hidden_dims={dims}")
                model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer, hidden_dim=dims)
                optimizer = torch.optim.Adam(model.parameters(), lr=LR)
                train_model(model, melody, harmonies, optimizer, criterion, epoch)
                with torch.no_grad():
                    output = model(melody)
                    loss = criterion(output, harmonies)
                    print(f"Final Loss: {loss.item()}")        
                # Keep track of the best model (with lowest loss)
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch, 'hidden_dim': dims}
print("BEST: ", best_params)

In [195]:
def inverse_df(df):
    X_scaled = df * 127
    return X_scaled

def midi_to_note(part):
    result = stream.Part()
    count = 1
    prev = round(part[0])
    for i in range(1, len(part)):
        pitch = part[i]
        curr = round(pitch)
        if curr == prev:
            count += 1
        else:
            this_note = note.Note(prev, quarterLength=count/4)
            result.append(this_note)
            count = 1
        prev = curr
    this_note = note.Note(prev, quarterLength=count/4)
    result.append(this_note)
    return result

def output_to_sheet_music(melody, result):
    result_numpy = result.detach().numpy()
    melody = inverse_df(melody).squeeze()
    inversed = inverse_df(np.squeeze(result_numpy)).T
    
    score = stream.Score()
    melody_part = midi_to_note(melody.numpy())
    
    alto_notes = inversed[0]
    tenor_notes = inversed[1]
    bass_notes = inversed[2]  
    
    alto_part = midi_to_note(alto_notes)
    tenor_part = midi_to_note(tenor_notes)
    bass_part = midi_to_note(bass_notes)

    score.append(melody_part)
    score.append(alto_part)
    score.append(tenor_part)
    score.append(bass_part)
    score.show('midi')
    score.write('musicxml', 'output.xml')

In [409]:
test_song = test[0]
melody = test_song.iloc[0]

# harmonies_with_zero = torch.zeros(1, 180, 3)
melody = torch.tensor(test_song.iloc[0].values.reshape(-1,1), dtype=torch.float32).unsqueeze(0)
harmonies = torch.tensor(test_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
model_input = torch.cat((melody, harmonies_with_zero), dim = -1)
songgg = torch.tensor(test_song.values.T, dtype=torch.float32).unsqueeze(0)
result = model(songgg)
output_to_sheet_music(melody, result)

<music21.stream.Part 0x15f33bed0>


In [140]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??

In [ ]:
# real one and generated compare
# train on all songs + test on a different song
# measure the test loss not just the training loss

In [ ]:
[60, 0, 0, 0] -> [60, 70, 70, 70] -> [61, ]